In [23]:
import numpy as np
import pandas as pd
import tsplib95

In [42]:
def load_path(file_path):
    '''Load the optimal route.'''
    dic = tsplib95.load(file_path)
    return dic.tours[0]

optimal_route = load_path("TSP-Configurations\eil51.opt.tour.txt")
print(optimal_route)

def load_file(file_path): 
    '''Load the coordinates of each dimension.'''
    dic = tsplib95.load(file_path)

    x_coordinates = [coords[0] for _, coords in list(dic.node_coords.items())]
    y_coordinates = [coords[1] for _, coords in list(dic.node_coords.items())]
    
    return np.array(x_coordinates), np.array(y_coordinates)

x_coordinates, y_coordinates = load_file("TSP-Configurations\eil51.tsp.txt")
print(x_coordinates)
print(y_coordinates)

[1, 22, 8, 26, 31, 28, 3, 36, 35, 20, 2, 29, 21, 16, 50, 34, 30, 9, 49, 10, 39, 33, 45, 15, 44, 42, 40, 19, 41, 13, 25, 14, 24, 43, 7, 23, 48, 6, 27, 51, 46, 12, 47, 18, 4, 17, 37, 5, 38, 11, 32]
[37 49 52 20 40 21 17 31 52 51 42 31  5 12 36 52 27 17 13 57 62 42 16  8
  7 27 30 43 58 58 37 38 46 61 62 63 32 45 59  5 10 21  5 30 39 32 25 25
 48 56 30]
[52 49 64 26 30 47 63 62 33 21 41 32 25 42 16 41 23 33 13 58 42 57 57 52
 38 68 48 67 48 27 69 46 10 33 63 69 22 35 15  6 17 10 64 15 10 39 32 55
 28 37 40]


In [61]:
def get_distance(route, x_coordinates, y_coordinates): 
    '''Calculate the total distance with given route.'''
    distance = 0
    for idx, i in enumerate(route[1:]): 
        x_distance = np.abs(x_coordinates[i-1] - x_coordinates[route[idx]-1])
        y_distance = np.abs(y_coordinates[i-1] - y_coordinates[route[idx]-1])
        distance += np.sqrt(x_distance**2 + y_distance**2)
    x_distance =  np.abs(x_coordinates[route[-1]-1] - x_coordinates[route[0]-1])
    y_distance =  np.abs(y_coordinates[route[-1]-1] - y_coordinates[route[0]-1])
    distance += np.sqrt(x_distance**2 + y_distance**2)
    
    return distance

distance = get_distance(optimal_route, x_coordinates, y_coordinates)
distance_iter = get_distance((route+np.ones(51)).astype(int), x_coordinates, y_coordinates)
print(optimal_route)
print(distance)
print(optimal_route[-1])
print(distance_iter)

[1, 22, 8, 26, 31, 28, 3, 36, 35, 20, 2, 29, 21, 16, 50, 34, 30, 9, 49, 10, 39, 33, 45, 15, 44, 42, 40, 19, 41, 13, 25, 14, 24, 43, 7, 23, 48, 6, 27, 51, 46, 12, 47, 18, 4, 17, 37, 5, 38, 11, 32]
429.98331198338406
32
491.05443237990494


In [26]:
def get_distance_matrix(x_coordinates, y_coordinates): 
    dimension = len(x_coordinates)
    distance_matrix = np.zeros((dimension, dimension))
    for i in range(dimension): 
        distance_matrix[i] = (x_coordinates - x_coordinates[i])**2
        distance_matrix[i] += (y_coordinates - y_coordinates[i])**2
    
    return np.sqrt(distance_matrix)


In [27]:
distance_matrix = get_distance_matrix(x_coordinates, y_coordinates)

In [28]:
distance_matrix

array([[ 0.        , 12.36931688, 19.20937271, ..., 26.40075756,
        24.20743687, 13.89244399],
       [12.36931688,  0.        , 15.29705854, ..., 21.02379604,
        13.89244399, 21.02379604],
       [19.20937271, 15.29705854,  0.        , ..., 36.22154055,
        27.29468813, 32.55764119],
       ...,
       [26.40075756, 21.02379604, 36.22154055, ...,  0.        ,
        12.04159458, 21.63330765],
       [24.20743687, 13.89244399, 27.29468813, ..., 12.04159458,
         0.        , 26.17250466],
       [13.89244399, 21.02379604, 32.55764119, ..., 21.63330765,
        26.17250466,  0.        ]])

In [ ]:
optimal_route_ = (optimal_route - np.ones(51)).astype(int)
reshape_route = np.array((optimal_route_[:-1], optimal_route_[1:])).T
distance_matrix[reshape_route[:, 0], reshape_route[:, 1]].sum()

421.1148337329819

In [29]:
rng = np.random.default_rng()
random_path = rng.choice(np.arange(1, 52), size=51, replace=False)

distance = get_distance(random_path, x_coordinates, y_coordinates)
print(distance)

1746.4935162039571


In [53]:
class SA(): 
    def __init__(self, dimension, temperature, num_i):
        if dimension == 51: 
            self.dimension = 51
            self.x_coordinates, self.y_coordinates = load_file("TSP-Configurations\eil51.tsp.txt")
        elif dimension == 280:
            self.dimension = 280
            self.x_coordinates, self.y_coordinates = load_file("TSP-Configurations\a280.tsp.txt")
        elif dimension == 442: 
            self.dimension = 442
            self.x_coordinates, self.y_coordinates = load_file("TSP-Configurations\pcb442.tsp.txt")
        else: 
            raise ValueError('No such file')
        
        rng = np.random.default_rng()
        self.route = rng.choice(range(self.dimension), size=self.dimension, replace=False)
        self.distance_matrix = self.get_distance_matrix()
        self.distance = self.get_distance(self.route)
        self.T = temperature
        self.num_i = num_i

    def get_distance_matrix(self): 
        distance_matrix = np.zeros((self.dimension, self.dimension))
        for i in range(self.dimension): 
            distance_matrix[i] = (x_coordinates - x_coordinates[i])**2
            distance_matrix[i] += (y_coordinates - y_coordinates[i])**2
        
        return np.sqrt(distance_matrix)
    
    def get_distance(self, route):
        reshape_route = np.array((route[:-1], route[1:])).T

        return self.distance_matrix[reshape_route[:, 0], reshape_route[:, 1]].sum()
    
    def two_opt(self): 
        position_1, position_2 = np.sort(rng.choice(range(self.dimension), size=2, replace=False))
        new_route = np.copy(self.route)
        new_route[position_1:position_2+1] = self.route[np.arange(position_2, position_1-1, -1)]
        new_distance = self.get_distance(new_route) 

        return new_route, new_distance

    def acceptance_criteria(self, new_route, new_distance): 
        random = np.random.rand()
        if random <= min(np.exp(-(new_distance-self.distance)/self.T),1): 
            self.route = new_route
            self.distance = new_distance
    
    def cooling_schedule(self): 
        self.T -= 0.000001

    def run_simulation(self): 
        distance_list = np.zeros(self.num_i)
        for i in range(self.num_i): 
            new_route, new_distance = self.two_opt()
            self.acceptance_criteria(new_route, new_distance)
            self.cooling_schedule()
            distance_list[i] = self.distance
        
        return distance_list, self.route
    
simu = SA(dimension=51, temperature=1, num_i=1000000)
distance, route = simu.run_simulation()
distance


array([1597.87832444, 1586.5004499 , 1578.49493955, ...,  405.42149479,
        405.42149479,  405.42149479])

In [62]:
reshape_route

array([[35, 34],
       [34, 19],
       [19,  2],
       [ 2, 27],
       [27, 30],
       [30,  7],
       [ 7, 25],
       [25,  6],
       [ 6, 42],
       [42, 23],
       [23, 22],
       [22, 47],
       [47, 26],
       [26,  5],
       [ 5, 13],
       [13, 24],
       [24, 17],
       [17,  3],
       [ 3, 16],
       [16, 46],
       [46, 11],
       [11, 45],
       [45, 50],
       [50, 31],
       [31,  0],
       [ 0, 21],
       [21,  1],
       [ 1, 28],
       [28, 20],
       [20, 33],
       [33, 29],
       [29,  8],
       [ 8, 49],
       [49, 15],
       [15, 10],
       [10, 37],
       [37,  4],
       [ 4, 48],
       [48,  9],
       [ 9, 38],
       [38, 32],
       [32, 44],
       [44, 14],
       [14, 36],
       [36, 43],
       [43, 41],
       [41, 18],
       [18, 39],
       [39, 40],
       [40, 12]])